In [1]:
from __future__ import print_function
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from keras.utils.data_utils import get_file
from keras.layers.embeddings import Embedding
from keras import layers
from keras.layers import recurrent, Bidirectional, Dense, Flatten
from keras.layers import Activation,recurrent, Bidirectional, Dense, Flatten, Conv1D, Lambda
from keras.layers import Dropout, LSTM, GRU, concatenate, multiply, add, Reshape, MaxPooling1D, BatchNormalization
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras import backend as K
from keras.models import load_model
import json
from pprint import pprint
import numpy as np
import csv
import string
import re
import keras
from keras.layers.embeddings import Embedding
from keras import layers
from keras.engine.topology import Layer
from nltk.tokenize.moses import MosesDetokenizer, MosesTokenizer

Using TensorFlow backend.


In [2]:

def get_dictionary(vocab):
    d = dict()
    with open(vocab) as f:
        lines = f.readlines()
        for l in lines:
            values = l.strip().split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            d[word] = coefs
    return d

In [3]:
d = get_dictionary("glove/glove.6B.300d.txt")

In [4]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()
    
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [38]:
with open('test.json') as data_file:    
    data = json.load(data_file)

In [39]:
ids = []
titles = dict()
contexts = dict()
questions = dict()
answers_text = dict()
answers_start = dict()
for i in range(len(data)):
    paragraphs = data[i]["paragraphs"]
    title = data[i]["title"]
    for j in range(len(paragraphs)):
        context = paragraphs[j]["context"]
        qas = paragraphs[j]["qas"]
        for k in range(len(qas)):
#             answer = qas[k]["answer"]
            id_ = qas[k]["id"]
            question = qas[k]["question"]
            ids.append(id_)
            titles[id_] = title
            contexts[id_] = context
            questions[id_] = question
#             answers_start[id_] = answer["answer_start"]
#             answers_text[id_] = answer["text"]

In [40]:
max_para = 600
max_q = 50
dimension = 300

In [41]:
test_len = len(ids)
# test_len = 500

In [42]:
# In[167]:

paras = np.zeros((test_len, max_para, dimension))
qns = np.zeros((test_len, max_q, dimension))
exact_match = np.zeros((test_len, max_para, 3))
lmtzr = WordNetLemmatizer()


# In[168]:

for i in range(test_len):
    if i % 5000 == 0:
        print(i)
    words = word_tokenize(contexts[ids[i]])
    qs = word_tokenize(questions[ids[i]])
    qs_1 = [w.lower() for w in qs]
    qs_2 = [lmtzr.lemmatize(w) for w in qs_1]
    for j in range(min(max_para - 1, len(words))):
        if words[j].lower() in d:
            paras[i][j] = d[words[j].lower()]
        if words[j] in qs:
            exact_match[i][j][0] = 1
        if words[j].lower() in qs_1:
            exact_match[i][j][1] = 1
        if lmtzr.lemmatize(words[j].lower()) in qs_2:
            exact_match[i][j][2] = 1
    for j in range(min(max_q - 1, len(qs))):
        if qs[j].lower() in d:
            qns[i][j] = d[qs[j].lower()]


0
5000
10000
15000
20000
25000
30000
35000


In [43]:

class MyLayer(Layer):

    def __init__(self, output_dim, name, **kwargs):
        self.output_dim = output_dim
        self.name = name
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(name=self.name,
                                      shape=(input_shape[2], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        return K.dot(x, self.W)

    def compute_output_shape(self, input_shape):
        return (input_shape[0],input_shape[1], self.output_dim)


# In[13]:

P = layers.Input(shape=(max_para,dimension), dtype='float32')
Q = layers.Input(shape=(max_q,dimension), dtype='float32')
P_exact_match = layers.Input(shape=(max_para,3), dtype='float32')


# In[14]:

alpha_w = MyLayer(128,"alpha_w")
alpha_l = MyLayer(128,"alpha_l")
alpha_h = MyLayer(128,"alpha_h")
alpha_u = MyLayer(128,"alpha_u")
alpha_s = MyLayer(128,"alpha_s")
beta = MyLayer(1,"beta")
gamma_s = MyLayer(250, "gamma_s")
gamma_e = MyLayer(250, "gamma_e")
hidden_unit = 125


# In[15]:

# Word level fusion
aligned_P = layers.Activation("relu")(alpha_w(P))
aligned_Q = layers.Activation("relu")(alpha_w(Q))
word_fusion = layers.dot([aligned_Q, aligned_P], axes=2)
word_fusion = Reshape((max_para * max_q,))(word_fusion)
word_fusion = layers.Activation("softmax")(word_fusion)
word_fusion = Reshape((max_q, max_para,))(word_fusion)
word_fusion = layers.dot([word_fusion,Q],axes=1)


# READING
final_P = layers.concatenate([P, P_exact_match, word_fusion], axis=2)
final_P = Dropout(.4)(final_P)
low_P = Bidirectional(LSTM(hidden_unit,return_sequences=True))(final_P)
low_P = Dropout(.4)(low_P)
high_P = Bidirectional(LSTM(hidden_unit,return_sequences=True))(low_P)
high_P = Dropout(.4)(high_P)

final_Q = Dropout(.4)(Q)
low_Q = Bidirectional(LSTM(hidden_unit,return_sequences=True))(final_Q)
low_Q = Dropout(.4)(low_Q)
high_Q = Bidirectional(LSTM(hidden_unit,return_sequences=True))(low_Q)
high_Q = Dropout(.4)(high_Q)


# Question understanding
U_Q = layers.concatenate([low_Q, high_Q], axis=2)
U_Q = Bidirectional(LSTM(hidden_unit,return_sequences=True))(U_Q)
U_Q = Dropout(.4)(U_Q)
w = beta(U_Q)
UQ = layers.dot([w,U_Q],axes=1)

# History of words
HP = layers.concatenate([P, low_P, high_P], axis = 2)
HQ = layers.concatenate([Q, low_Q, high_Q], axis = 2)

# Low level fusion
low_HP = layers.Activation("relu")(alpha_l(HP))
low_HQ = layers.Activation("relu")(alpha_l(HQ))
low_fusion = layers.dot([low_HQ, low_HP], axes=2)
low_fusion = Reshape((max_para * max_q,))(low_fusion)
low_fusion = layers.Activation("softmax")(low_fusion)
low_fusion = Reshape((max_q, max_para,))(low_fusion)
low_fusion = layers.dot([low_fusion,low_Q],axes=1)
low_fusion = Dropout(.4)(low_fusion)

# High level fusion
high_HP = layers.Activation("relu")(alpha_h(HP))
high_HQ = layers.Activation("relu")(alpha_h(HQ))
high_fusion = layers.dot([high_HQ, high_HP], axes=2)
high_fusion = Reshape((max_para * max_q,))(high_fusion)
high_fusion = layers.Activation("softmax")(high_fusion)
high_fusion = Reshape((max_q, max_para,))(high_fusion)
high_fusion = layers.dot([high_fusion,high_Q],axes=1)
high_fusion = Dropout(.4)(high_fusion)

# Understanding level fusion
U_HP = layers.Activation("relu")(alpha_u(HP))
U_HQ = layers.Activation("relu")(alpha_u(HQ))
U_fusion = layers.dot([U_HQ, U_HP], axes=2)
U_fusion = Reshape((max_para * max_q,))(U_fusion)
U_fusion = layers.Activation("softmax")(U_fusion)
U_fusion = Reshape((max_q, max_para,))(U_fusion)
U_fusion = layers.dot([U_fusion,U_Q],axes=1)
U_fusion = Dropout(.4)(U_fusion)

# Fully-aware attention
P_fusion = layers.concatenate([low_P, high_P, low_fusion, high_fusion, U_fusion], axis=2)
V_P = Bidirectional(LSTM(hidden_unit,return_sequences=True))(P_fusion)
V_P = Dropout(.4)(V_P)
H_P = layers.concatenate([P, P_fusion, V_P], axis=2)

# Self-boosted fusion
self_HP = layers.Activation("relu")(alpha_s(H_P))
self_fusion = layers.dot([self_HP, self_HP], axes=2)
self_fusion = Reshape((max_para * max_para,))(self_fusion)
self_fusion = layers.Activation("softmax")(self_fusion)
self_fusion = Reshape((max_para, max_para))(self_fusion)
self_fusion = layers.dot([self_fusion,V_P],axes=1)
self_fusion = Dropout(.4)(self_fusion)

U_P = layers.concatenate([V_P, self_fusion], axis=2)
U_P = Bidirectional(LSTM(hidden_unit,return_sequences=True))(U_P)
U_P = Dropout(.4)(U_P)

start = gamma_s(U_P)
start = layers.dot([start, UQ], axes=2)
start = Reshape((max_para,))(start)
start = layers.Activation("softmax")(start)

end = gamma_e(U_P)
end = layers.dot([end, UQ], axes=2)
end = Reshape((max_para,))(end)
end = layers.Activation("softmax")(end)


In [44]:
model = Model([P, Q, P_exact_match],[start, end])
model.load_weights('fusion.h5')

In [45]:
out = model.predict([paras,qns,exact_match], verbose=1, batch_size=64)

36790/36790 [==============================] - 7099s 193ms/step


In [46]:
out_1 = np.argmax(out[0], axis=1)
out_2 = np.argmax(out[1], axis=1)

In [47]:
def search(para, pos):
    word_list = word_tokenize(para)
    s = ''.join(word_list[:pos])
    if pos == 0:
        return 0
    if len(word_list) <= pos:
        return len(para)
    else:
        begin, end = 0, len(para)
        c = 0
        while begin < end and c < 100:
            mid = (begin + end) // 2
            c += 1
            a = word_tokenize(para[:mid])
            s_ = ''.join(a)
            if (len(s_) < len(s)):
                begin = mid + 1
            elif (len(s_) > len(s)):
                end = mid
            elif s == s_:
                break
        return mid

In [48]:
for i in range(test_len):
    s = 0
    t = 1
    if i%5000 == 0: 
        print(i)
    l = len(word_tokenize(contexts[ids[i]]))
    for j in range(min(l, max_para)):
        for k in range(j, min(j+15, min(l, max_para))):
            if out[0][i][s] * out[1][i][t] < out[0][i][j] * out[1][i][k]:
                s = j
                t = k
    out_1[i] = s
    out_2[i] = t
#     print(s, t)

0
5000
10000
15000
20000
25000
30000
35000


In [49]:
out_2 = out_2 + 1

In [50]:
# c = 0
# for i in range(5000):
#     a = search(contexts[ids[i]], out_1[i], out_2[i])
#     if normalize_answer(a) == normalize_answer(answers_text[ids[i]]):
#         c += 1
# print(c/5000)

In [51]:

with open("fusion.csv", 'w') as f:
    writer = csv.writer(f, delimiter=',')
    writer.writerow(['Id','Answer'])
    for i in range(test_len):
        a = search(contexts[ids[i]], out_1[i])
        b = search(contexts[ids[i]], out_2[i])
        writer.writerow([ids[i],normalize_answer(contexts[ids[i]][a:b])])
        # writer.writerow([ids[i],normalize_answer(contexts[ids[i]][a:b]).encode('utf-8')])
        if i%5000 == 0: 
            print(i)

0
5000
10000
15000
20000
25000
30000
35000
